# Plan 
## Simulating the sensors
## Importing/creating the models
## Implementing the anomaly detection

In [3]:
import pandas as pd
import numpy as np
import os
ORIGIN_PATH = os.getcwd()

In [4]:
def getSubjectNum(fileName) :
        import re
        n = re.findall(r'\d+', fileName)
        return int(n[0])


In [5]:
class acceleration_chest_X :
     id =0
     name = 'acceleration_chest_X'

class acceleration_chest_Y :
     id =1
     name = 'acceleration_chest_Y'

class acceleration_chest_Z :
     id =2
     name = 'acceleration_chest_Z'

class ECG_lead_1 :
     id=3
     name = 'ECG_lead_1'

class ECG_lead_2 :
     id=4
     name = 'ECG_lead_2'

class acceleration_left_ankle_X :
     id =5
     name = 'acceleration_left_ankle_X'

class acceleration_left_ankle_Y :
     id =6
     name = 'acceleration_left_ankle_Y'

class acceleration_left_ankle_Z :
     id =7
     name = 'acceleration_left_ankle_Z'

class gyro_left_ankle_X :
     id =8
     name = 'gyro_left_ankle_X'

class gyro_left_ankle_Y :
     id =9
     name = 'gyro_left_ankle_Y'

class gyro_left_ankle_Z :
     id =10
     name = 'gyro_left_ankle_Z'

class magnetometer_left_ankle_X :
     id =11
     name = 'magnetometer_left_ankle_X'

class magnetometer_left_ankle_Y :
     id =12
     name = 'magnetometer_left_ankle_Y'

class magnetometer_left_ankle_Z :
     id =13
     name = 'magnetometer_left_ankle_Z'

class acceleration_right_lower_arm_X :
     id =14
     name = 'acceleration_right_lower_arm_X'

class acceleration_right_lower_arm_Y :
     id =15
     name = 'acceleration_right_lower_arm_Y'

class acceleration_right_lower_arm_Z :
     id =16
     name = 'acceleration_right_lower_arm_Z'

class gyro_right_lower_arm_X :
     id =17
     name = 'gyro_right_lower_arm_X'

class gyro_right_lower_arm_Y :
     id =18
     name = 'gyro_right_lower_arm_Y'

class gyro_right_lower_arm_Z :
     id =19
     name = 'gyro_right_lower_arm_Z'

class magnetometer_right_lower_arm_X :
     id =20
     name = 'magnetometer_right_lower_arm_X'

class magnetometer_right_lower_arm_Y :
     id =21
     name = 'magnetometer_right_lower_arm_Y'


class magnetometer_right_lower_arm_Z :
     id =22
     name = 'magnetometer_right_lower_arm_Z'
    

In [6]:
class mHealthSensor:
    def __init__(self,TYPE,freq=50,sourceData="mHealth_subject3.log",realisitc=False):
        self.frequency=freq
        self.sourceData=sourceData
        self.column=TYPE.id
        self.name=TYPE.name
        self.realisitc=realisitc
        self.subjectNum=getSubjectNum(self.sourceData)
        self.generateInputFile()
        self.datapool=pd.DataFrame(np.array([]))
        
        
    
    def generateInputFile(self):
        X =  np.array([])
        Y =  np.array([])
        df = pd.read_csv(self.sourceData, header=None, delim_whitespace=True)
        df = df[df[23]!=0] #drop 0 labeled features
        readings_df = pd.DataFrame(df[self.column].copy())
        targets_df = pd.DataFrame(df[23].copy())
        reading_np = readings_df.values

        rps=self.frequency
        activities=range(0,13)
        
        # #         segmentation
        for i in range(int(readings_df.shape[0]/rps)):
            if  float(targets_df[i*rps:i*rps+rps].mean()) in activities:
                Y=np.append(Y,targets_df[i*rps:i*rps+rps].mean()) #new
                X=np.append(X,readings_df[i*rps:i*rps+rps].values)

        X=X.reshape(-1,rps)
        OutDF = pd.DataFrame(X)
        OutDF['labels']=Y
        OutDF.reset_index
        OutDF.to_csv(self.name+"_subject"+str(self.subjectNum), sep='\t',index=False)

    #returns value , target
    def getReading(self,activity=1):
        if self.realisitc:
            import time
            time.sleep(1)
        if self.datapool.empty == True:
            self.datapool= pd.read_csv(self.name+"_subject"+str(self.subjectNum), sep='\t',index_col=False)
        readings = self.datapool.loc[self.datapool.labels==activity,]
        randIndex = np.random.randint(0,len(readings))
        return np.array(readings.iloc[randIndex,range(0,self.frequency)]), readings.iloc[randIndex,self.frequency]


In [7]:
os.chdir(ORIGIN_PATH)
source = os.path.join(ORIGIN_PATH,"MHEALTHDATASET","mHealth_subject3.log")
gyro = mHealthSensor(TYPE=gyro_left_ankle_X,freq=10,sourceData=source,realisitc=True)
ECG  =mHealthSensor(TYPE=ECG_lead_1,freq=50,sourceData=source,realisitc=True)

In [8]:
ECG.getReading()

(array([-0.22606  , -0.24699  , -0.38095  ,  2.3234   ,  0.69074  ,
        -0.18838  , -0.29723  , -0.2763   , -0.25118  , -0.23862  ,
        -0.1842   , -0.19676  , -0.20931  , -0.26374  , -0.3349   ,
        -0.35583  , -0.19676  , -0.0083726,  0.16327  ,  0.28885  ,
         0.3977   ,  0.4898   ,  0.60701  ,  0.70748  ,  0.83726  ,
         0.95029  ,  1.1177   ,  1.2098   ,  1.3564   ,  1.4819   ,
         1.641    ,  1.7122   ,  1.7499   ,  1.8127   ,  1.8629   ,
         1.8085   ,  4.4919   ,  2.8174   ,  2.0638   ,  2.001    ,
         1.9717   ,  1.9299   ,  1.9634   ,  1.9801   ,  1.9131   ,
         1.8336   ,  1.6871   ,  1.528    ,  1.4359   ,  1.4526   ]), 1.0)

# Trainers

### Download from cloud

In [9]:
from google_drive_downloader import GoogleDriveDownloader as gdd
if not os.path.isfile(os.path.join("MODELS","ecg.sav")):
    gdd.download_file_from_google_drive(file_id='1ZKUsE9l-iHZ7_z7OKUpcG6BbiJhKsdy0',
                                    dest_path=os.path.join("MODELS","ecg.sav"))
if not os.path.isfile(os.path.join("MODELS","knn.sav")):
    gdd.download_file_from_google_drive(file_id='1WpZuJLhcMaI40DWGVPeJyijSWSsWngze',
                                    dest_path=os.path.join("MODELS","knn.sav"))

In [10]:
from sklearn.externals import joblib
ActuatorsPredictor = joblib.load(os.path.join("MODELS","knn.sav"))
ECGPredictor = joblib.load(os.path.join("MODELS","ecg.sav"))

## Threading

In [11]:
import threading
import time
import os
import random
import keyboard
from sklearn.externals import joblib
from IPython.display import clear_output, display
ACTUATOR_BUFFER=[]
ECG_BUFFER=[]
os.chdir(ORIGIN_PATH)
source = os.path.join(ORIGIN_PATH,"MHEALTHDATASET","mHealth_subject3.log")
gyroSensor = mHealthSensor(TYPE=gyro_left_ankle_X,freq=50,sourceData=source,realisitc=True)
ECGSensor  = mHealthSensor(TYPE=ECG_lead_1,freq=50,sourceData=source,realisitc=True)
ActuatorsPredictor=joblib.load(os.path.join("MODELS","knn.sav"))
ECGPredictor=joblib.load(os.path.join("MODELS","ecg.sav"))

In [44]:
def actuatorsThread(ACTUATOR_WRAPPER):
    while True:
        time.sleep(0.5)
        clear_output(wait=True)
        if keyboard.is_pressed('2'):
            ACTUATOR_WRAPPER[0],_=gyroSensor.getReading()
            print("Actuator sensor detected: ",len(ACTUATOR_WRAPPER[0]))
        else:
            print("Actuator sensor NOT DETECTED")
        time.sleep(0.5)
        if keyboard.is_pressed('t'):
            return
    

In [43]:
def ECGThread(ECG_BUFFER):
    while True:
        time.sleep(0.5)
        clear_output(wait=True)
        if keyboard.is_pressed('1'):
            ECG_BUFFER[0],_=ECGSensor.getReading()
            print("ECG sensor detected :", len(ECG_BUFFER[0]))
        else:
            print("ECG sensor NOT DETECTED")
        time.sleep(0.5)
        if keyboard.is_pressed('t'):
            return

ECG sensor NOT DETECTED


In [45]:
ACTUATOR_BUFFER=[]
ECG_BUFFER=[]
ACTUATOR_WRAPPER=[ACTUATOR_BUFFER]
ECG_WRAPPER=[ECG_BUFFER]
import threading
t1 = threading.Thread(target=actuatorsThread,args=(ACTUATOR_WRAPPER,)) 
t2 = threading.Thread(target=ECGThread,args=(ECG_WRAPPER,)) 
# starting thread 1 
t1.start() 
# starting thread 2 
t2.start() 

# # wait until thread 1 is completely executed 
# t1.join() 
# # wait until thread 2 is completely executed 
# t2.join() 


Actuator sensor NOT DETECTED
ECG sensor NOT DETECTED


In [51]:
a = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
ECGPredictor.predict(np.array(a).reshape(-1, 1))

ValueError: Expected 2D array, got 1D array instead:
array=[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [49]:
t=0
while(True):
    activity=np.random.randint(1,13)
    time.sleep(1)
    if (len(ECG_WRAPPER[0])>0) and (len(ACTUATOR_WRAPPER[0])>0):
        Activity_predited_ECG = ECGPredictor.predict(np.array(ECG_WRAPPER[0]).reshape(-1, 1))
        Activity_predited_GYR = ActuatorsPredictor.predict(np.array(ACTUATOR_WRAPPER[0]).reshape(-1, 1))
        print(Activity_predited_ECG,'<-E-|',activity,'|-A->',Activity_predited_GYR)
        if Activity_predited_ECG == Activity_predited_GYR:
            print(">> ", t)
            t+=1
        time.sleep(1)
    else:
        print("No readings :",len(ECG_WRAPPER[0]),", ",len(ACTUATOR_WRAPPER[0]) )

ValueError: query data dimension must match training data dimension

In [177]:
t=0
tot=10000
for i in range(tot):
    x,y = G.getReading(np.random.randint(1,13))
    x = np.array(x).reshape(-1,50)
    if(y==knn.predict(x)):
        t+=1
print(t/tot)

0.5436


In [ ]:
def ECG_Reader():
    pass

In [ ]:
def Gyro_Reader():
    pass

In [1]:

def fun1():
      for j in range(10):
            print("1")
            time.sleep(random.randint(1,5)/10)

def fun2():
      for i in range(10):
            print("2")
            time.sleep(random.randint(5,10)/10)

if __name__ == "__main__": 
    # creating thread 
    t1 = threading.Thread(target=fun1) 
    t2 = threading.Thread(target=fun2) 
  
    # starting thread 1 
    t1.start() 
    # starting thread 2 
    t2.start() 
  
    # wait until thread 1 is completely executed 
    t1.join() 
    # wait until thread 2 is completely executed 
    t2.join() 
  
    # both threads completely executed 
    print("Done!") 

1
2
1
1
2
1
1
2
1
1
2
1
1
2
1
2
2
2
2
2
Done!


In [6]:
a = input()
print(a)

asd
asd
